In [30]:
%cd

C:\Users\YeChan


In [45]:
import cv2
import sys
import os
import time
from datetime import datetime

In [41]:
def get_unique_filename(base_path):
    ### 파일 저장 경로에 이미 파일이 존재하면 새로운 파일명 제공
    
    folder, fname = os.path.split(base_path)
    name, ext = os.path.splitext(fname)
    
    candidate = base_path
    counter = 1
    while os.path.exists(candidate):
        candidate = os.path.join(folder, f"{name}{counter}{ext}")
        counter += 1
    return candidate


def put_shadow_text(img, text, org, font, scale, color, thick):
    ### 배경에 관계 없는 text
    cv2.putText(img, text, (org[0]+1, org[1]+1), font, scale, (0,0,0), thick+2, cv2.LINE_AA)
    cv2.putText(img, text, org, font, scale, color, thick, cv2.LINE_AA)

In [50]:
##### OUTPUT DIR #####
out_dir = "./CV_HW_1"
os.makedirs(out_dir, exist_ok=True)
base_path = os.path.join(out_dir, "output.avi")
out_path = get_unique_filename(base_path)

print("최종 저장 경로:", out_path)
######################

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Camera open failed!")
    sys.exit()

w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'DIVX')

delay = round(1000/fps)

out = cv2.VideoWriter(out_path, fourcc, fps, (w,h))

### 기본 세팅값
recording = True
preview_gray = False
show_grid = False
start_time = time.time()
rec_start_time = time.time()
frame_count = 0

while True:
    ret, frame = cap.read() # 불러오기 시작

    if not ret: # 예외시 멈춤
        break
    
    key = cv2.waitKey(delay) # 키보드 값 받아오기

    if key == 27: # ESC 키 종료
        break
    elif key == 32: # SPACE 모드 전환
        recording = not recording
    elif key == ord('g'):
            preview_gray = not preview_gray
    elif key == ord('l'):
            show_grid = not show_grid
    elif key == ord('s'): # 스크린샷 저장
            ts = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]
            shot = os.path.join(out_dir, f"shot_{ts}.png")
            cv2.imwrite(shot, frame)
            print(f"[SHOT] {shot}")
    if recording:
        out.write(frame)

    im0 = frame.copy()
    if preview_gray:
            im0 = cv2.cvtColor(im0, cv2.COLOR_BGR2GRAY)
            im0 = cv2.cvtColor(im0, cv2.COLOR_GRAY2BGR)

    #################
    ### Text info ###
    center = (20, 20)           # 좌상단 기준 위치
    radius = 8
    red = (0, 0, 255)           # BGR
    gray = (180, 180, 180)
    text_pos = (40, 27)         # 'REC' 텍스트 위치
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.6
    thickness = 2
    #################
    #################

    if recording:
        frame_count += 1
        cv2.circle(frame, center, radius, red, -1)
        put_shadow_text(im0, "REC", (40, 27), cv2.FONT_HERSHEY_SIMPLEX, 0.6, red, 2)
    else:
        frame_count = 0
        cv2.circle(frame, center, radius, gray, 2)
        put_shadow_text(im0, "Pause", (40, 27), cv2.FONT_HERSHEY_SIMPLEX, 0.6, gray, 2)

    now = time.time()
    rec_dur = now - rec_start_time if recording else 0
    total_dur = now - start_time
    
    info = f"{'REC' if recording else 'PAUSE'} | seg {int(rec_dur)}s | total {int(total_dur)}s | frame {frame_count}"
    put_shadow_text(im0, info, (10, h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
    
    if show_grid:
        rows, cols = 3, 3
        H, W = im0.shape[:2]
        for c in range(1, cols):
            x = int(W * c / cols); cv2.line(im0, (x,0), (x,H), (200,200,200), 1)
        for r in range(1, rows):
            y = int(H * r / rows); cv2.line(im0, (0,y), (W,y), (200,200,200), 1)

    cv2.imshow('frame', im0)

    

cap.release()
out.release()
cv2.destroyAllWindows()


최종 저장 경로: ./CV_HW_1\output2.avi
